2.Crawl pages whose seed url is the press releases page of the Securities and Exchange Commission: https://www.sec.gov/news/pressreleases and collect urls of press releases that contain the word "charges". The code should output the first 20 such links that it finds. For each link output the url and the text. Similar to the previous task, when checking for the presence of the word "charges" you should consider lower- and upper-case versions.

In [1]:
import urllib.request
from bs4 import BeautifulSoup
import requests

# Crawl 10 Pages with word "covid" and store in List
seed_url = "https://www.sec.gov/news/pressreleases"
domain = "https://www.sec.gov/news"

urls = [seed_url]  # queue of urls to crawl
seen = [seed_url]  # stack of urls seen so far
opened = []
charges_urls = []

maxNumUrl = 20;  # set the maximum number of urls included 'covid' to visit
print("Starting with url=" + str(urls))
while len(urls) > 0 and len(charges_urls) < maxNumUrl:
    # DEQUEUE A URL FROM urls AND TRY TO OPEN AND READ IT
    try:
        curr_url = urls.pop(0)  # pop：delete the first item of the list
        print("num. of URLs in stack: %d " % len(urls))
        print("Trying to access= " + curr_url)
        req = urllib.request.Request(curr_url, headers={'User-Agent': 'Mozilla/5.0'}) 
        webpage = urllib.request.urlopen(req).read()
        # print(webpage)
        opened.append(curr_url)

    except Exception as ex:
        print("Unable to access= " + curr_url)
        print(ex)
        continue  # skip code below

    # IF URL OPENS, CHECK WHICH URLS THE PAGE CONTAINS
    # ADD THE URLS FOUND TO THE QUEUE url AND seen
    soup = BeautifulSoup(webpage)  # creates object soup 

    for tag in soup.find_all('a', href=True):  # find tags with links
        childUrl = tag['href']  # extract just the link
        childUrl = urllib.parse.urljoin(seed_url, childUrl)  # join two urls
        if domain in childUrl and childUrl not in seen:
            urls.append(childUrl)
            seen.append(childUrl)

            # search 'charges' in the press release text
            response = requests.get(childUrl)
            soup_child = BeautifulSoup(response.text, 'html.parser')
            keyword_found = 0
            # gather title, release number, location, publish date, and article body.
            article = soup_child.find_all('h1', class_='article-title')+soup_child.find_all('p', class_='release-number')+soup_child.find_all('p', class_='article-location-publishdate')+soup_child.find_all('div', class_='article-body')
            if len(article): # this webpage contains an article (press release)
                for item in article:
                    if 'charges' in item.text.lower():
                        keyword_found = 1
                        break
            else:
                continue

            if keyword_found:
                charges_urls.append(childUrl)
                print('# '+str(len(charges_urls))+': '+ childUrl +"\n")

                text_output_path = './press_releases_that_contain_CHARGES/'+str(len(charges_urls))+'.txt'
                
                # put the URL at the beginning of the output file
                output_text = [childUrl+'\n',] 
                
                for item in article:
                    output_text.append(item.text)
                # print the press releases that contain 'charges' to seperate .txt files
                with open(text_output_path, "w+", encoding='utf-8') as foutput: 
                    foutput.writelines(output_text)

                if len(charges_urls) >= maxNumUrl:
                    break

print("num. of URLs seen = %d, and scanned = %d" % (len(seen), len(opened)))

# Output 20 links with word "covid" in csv file
import pandas as pd
charges_links = pd.DataFrame(charges_urls)
charges_links.columns = ['charges_links']
charges_links.to_csv('charges_links.csv',mode='w+',header=True, index=None)

Starting with url=['https://www.sec.gov/news/pressreleases']
num. of URLs in stack: 0 
Trying to access= https://www.sec.gov/news/pressreleases
# 1: https://www.sec.gov/news/sec-webcasts

# 2: https://www.sec.gov/news/press-release/2022-183

# 3: https://www.sec.gov/news/press-release/2022-182

# 4: https://www.sec.gov/news/press-release/2022-181

# 5: https://www.sec.gov/news/press-release/2022-180

# 6: https://www.sec.gov/news/press-release/2022-179

# 7: https://www.sec.gov/news/press-release/2022-178

# 8: https://www.sec.gov/news/press-release/2022-176

# 9: https://www.sec.gov/news/press-release/2022-175

# 10: https://www.sec.gov/news/press-release/2022-174

# 11: https://www.sec.gov/news/press-release/2022-173

# 12: https://www.sec.gov/news/press-release/2022-172

# 13: https://www.sec.gov/news/press-release/2022-171

# 14: https://www.sec.gov/news/press-release/2022-170

# 15: https://www.sec.gov/news/press-release/2022-169

# 16: https://www.sec.gov/news/press-release/2022-